In [1]:
from feret_utils import extract_faces_gs
import numpy as np
import cv2
import tensorflow as tf
import pickle
import time

In [2]:
# Get the tensorflow model
tf.logging.set_verbosity(tf.logging.INFO)
predict_fn = tf.contrib.predictor.from_saved_model("feret/trained/trained_0/1532617597")
mean_image, std_image = 103.659775, 52.517044

INFO:tensorflow:Restoring parameters from feret/trained/trained_0/1532617597/variables/variables


In [3]:
with open('known_faces/faces.pickle', 'rb') as handle:
    known_pics_dict = pickle.load(handle)
    
dense_dict = {}

for file, pic in known_pics_dict.items():
    for i, letter in enumerate(file):
        if not letter.isalpha():
            break
    name = file[:i]
    pic = (pic - mean_image)/std_image
    X = pic.reshape(96 * 96 * 1).flatten().tolist()
    model_input = tf.train.Example(features= tf.train.Features(feature={'x': tf.train.Feature(
                                    float_list=tf.train.FloatList(value=X))}))
    model_input = model_input.SerializeToString()
    output_dict = predict_fn({"predictor_inputs":[model_input]})
    dense_dict.setdefault(name, []).append(output_dict['dense'])

In [4]:
import pickle
# Save output_dict so we can unpickle it when needed
data_to_pickle = dense_dict
with open('known_faces/dense_dict.pickle', 'wb') as handle:
    pickle.dump(data_to_pickle, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [5]:
import pickle
# Unpickle data_dict
with open('known_faces/dense_dict.pickle', 'rb') as handle:
    dense_dict = pickle.load(handle)

In [6]:
def check_if_matched(img_dense, dense_dict):
    scores_dict = {}
    for name_itr in dense_dict.keys():   
        sum_score, score = 0, 0
        for dense_item in dense_dict[name_itr]:
            sum_score += np.mean((np.abs(img_dense - dense_item))**3)
        score = sum_score/len(dense_dict[name_itr])
        scores_dict[name_itr] = score
    lowest_name = min(scores_dict, key=scores_dict.get)
    return(lowest_name, scores_dict[lowest_name])

In [7]:
def create_dense_for_pic(img):
    img = (img - mean_image)/std_image
    X = img.reshape(96 * 96 * 1).flatten().tolist()
    model_input = tf.train.Example(features= tf.train.Features(feature={'x': tf.train.Feature(
                                    float_list=tf.train.FloatList(value=X))}))
    model_input = model_input.SerializeToString()
    dense_out = predict_fn({"predictor_inputs":[model_input]})
    
    return dense_out["dense"]

In [9]:
cap = cv2.VideoCapture(0)

face_cascade = cv2.CascadeClassifier('/usr/local/share/OpenCV/haarcascades/haarcascade_frontalface_default.xml')
found_time = 0
unknown_count = 0
while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Our operations on the frame come here
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor=1.09,
        minNeighbors=3,
        minSize=(96, 96)
    )
    if (len(faces) > 0):
        if time.time() - found_time > 1:
            positions = {}
            for (x, y, w, h) in faces:         
                crop_img = gray[y:y+h, x:x+w]
                crop_img = cv2.resize(crop_img, dsize=(96, 96), interpolation=cv2.INTER_NEAREST)
                dense_for_pic = create_dense_for_pic(crop_img)

                # Do predicition
                name, score = check_if_matched(dense_for_pic, dense_dict)
                print(score)
                if (score > 1):
                    name = ("Unknown")
                
                # Keep position data to continue drawing square when there is no prediction
                positions[(x, y, w, h)] = name
 
                cv2.rectangle(gray, (x, y), (x+w, y+h), (0, 255, 0), 2)
                cv2.putText(gray, name, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

                found_time = time.time()
        else:
            for pos, name in positions.items():
                x, y, w, h = pos
                cv2.rectangle(gray, (x, y), (x+w, y+h), (0, 255, 0), 2)
                cv2.putText(gray, name, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    # Display the resulting frame
    cv2.imshow('frame',gray)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

1.7872286319732666
0.5327664136886596
0.31440134048461915
0.8068335294723511
0.6208767652511596
0.6599600434303283
0.6654536008834839
0.544695895910263
0.8334670662879944
0.4289808630943298
0.4723049521446228
0.4445061504840851
7.865639019012451
7.172409152984619
0.5251597881317138
10.92102108001709
10.685856437683105
10.773926734924316
10.854878807067871
11.849378204345703
0.957095491886139
10.932256317138672
0.7191604971885681
12.083811187744141
11.031088066101074
10.922796630859375
11.094811820983887
10.554804229736328
0.48938660621643065
0.4351273238658905
0.45414906144142153
0.3961962997913361
0.46531933546066284
0.4254010558128357
0.437485271692276
0.40583178400993347
0.42464346885681153
0.40053836107254026
0.40163968205451966
0.4521890044212341
0.40141727328300475
0.4472260355949402
0.45241097211837766
0.4532576322555542
0.47786529660224913
0.5329801857471466
0.5926781892776489
10.91340446472168
0.8396038055419922
11.748898887634278
11.330296325683594
11.425756072998047
9.555648

In [8]:
cap = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier('/usr/local/share/OpenCV/haarcascades/haarcascade_frontalface_default.xml')
i = 0
while(True):

    # Capture frame-by-frame
    ret, frame = cap.read()
    i += 1
    # Our operations on the frame come here
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor=1.09,
        minNeighbors=3,
        minSize=(96, 96)
    )
    print(i)
    if (len(faces) == 1) and (i % 10 == 0):
        (x, y, w, h) = faces[0]
            
        crop_img = gray[y:y+h, x:x+w]
        crop_img = cv2.resize(crop_img, dsize=(96, 96), interpolation=cv2.INTER_NEAREST)
        cv2.imwrite(('samples/shahar_%s.jpg' % i),crop_img)
        
        
    
    # Display the resulting frame
    cv2.imshow('frame',gray)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    if i == 51:
        break
    
cap.release()
cv2.destroyAllWindows() 

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
